In [1]:
import tensorflow as tf
import numpy as np
import scipy as sp
import sklearn as skl
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import math
import os
from pathlib import Path
from tensorflow import keras
from keras import layers
from keras import models
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from datetime import datetime, date, timedelta

# Script
# - loads data from inidata.csv
# - generates training and test data sets for the FCNN + RNN predictive model.
# - Creates and trains a predictive model. Writes the learning history to the file 'history_p2.txt'

# Basic actions:
# 1. The test set is formed from the last 360 days of history simulated
# 2. To build time series of shipments, the script sorts history of shipments by fields: 
# customer type, customer, outbound location, supplier, 
# description group id, consisting of 3 descriptions that match in meaning but have different word orders, 
# shipment date. The script then groups shipments by the same fields, but without the shipment date.  
# Groups are placed in lists and combined with the history by the grouping fields.
# Those, in each row of the history (shipment) the new field appears with a list of all shimpents of the same group.  
# Each list is shortened to HISTORY_LENGTH of shipments that arrived at the warehouse before 
# the shipment/row to which the list belongs. Shipment-"owner" is not included in the final series. Rows in series contain 
# enhanced set of fields. Besids mentioned above fields RNN rows include: duration of shipmemt, not saled quantity, labels. 
# Since series consists of shipments till forecasting, we can use all shipments features. 
# A tensor with the time series in a separate dimension is formed.

WORKING_DIRECTORY = 'C:/Pilot/test/'
os.chdir (WORKING_DIRECTORY)

fpLog = Path ('log.txt')

with open (fpLog, 'w') as flog:
    print ('model p2 v1.3 starts at : ', datetime.now(), file = flog)

#fpini = Path ('intermediate.csv')  
fpini = Path ('inidata.csv')
dfdata = pd.read_csv (
    fpini, 
    dtype = {
        'tid':'float','cid':'float','oid':'float','sid':'float',
        'yy':'float','mm':'float','dd':'float','wd':'float',
        'is_i':np.int32,'txt':'str','nsl':'float','drn':'float','qnt':'float',
        'stop':'float','phash':np.int64,'T01':'float','T02':'float','T03':'float',
        'T04':'float','T05':'float','T06':'float','T07':'float','T08':'float',
        'T09':'float','T10':'float','T11':'float','T12':'float','T13':'float',
        'T14':'float','T15':'float','T16':'float','T17':'float','T18':'float',
        'T19':'float','T20':'float','did':'float',
        'tid_i':np.int32,'cid_i':np.int32,'oid_i':np.int32,'sid_i':np.int32,'did_i':np.int32
    }
    ).fillna(0)

#dfdata = pd.read_csv (fpini, dtype = 'float').fillna(0)

#dfdata['did_i'] = dfdata['did'].astype(np.int32)
#dfdid_stats = dfdata['did'].describe().transpose()

#dfdata['did'] = (dfdata['did'] - dfdid_stats['mean'])/dfdid_stats['std']

rng = np.random.default_rng()

ismax = np.max (dfdata.loc(axis = 1)['is_i'])
with open (fpLog, 'a') as flog:
    print ('ismax: ', ismax, file = flog)

dftest  = dfdata.loc[dfdata.is_i >= (ismax - 360)].copy()
dftrain = dfdata.loc[dfdata.is_i <  (ismax - 360)].copy()

with open (fpLog, 'a') as flog:
    print ('dftest: ',  dftest.shape,  file = flog)
    print ('dftrain: ', dftrain.shape, file = flog)

lsort = ['tid_i', 'cid_i', 'oid_i', 'sid_i', 'did_i', 'is_i']
lgb   = ['tid_i', 'cid_i', 'oid_i', 'sid_i', 'did_i']
lx1   = ['tid', 'cid', 'oid', 'sid', 'did', 'mm', 'dd', 'wd', 'qnt', 'stop']
lx2   = ['drn', 'nsl']
ly    = ["T%02d" % (i,) for i in range(1, 21)]
lxd   = lx1
lxa   = lx1 + lx2 + ly

# Full list of columns: lsort + lxd  + lxa + ly

HISTORY_LENGTH = 32
ROW_LENGTH = len (lxa)

def fnPrepareData (df):
    
    df.sort_values (by = lsort, inplace = True)

    dfgb = df.groupby(lgb)
    df['gn'] = dfgb.cumcount()
    dfgb = df.loc (axis = 1)[lxa + lgb].groupby (lgb)
    dflist = dfgb.agg (list)

    dfocean = df.merge (dflist, how = 'inner', on = lgb, suffixes = ["", "_l"])

    dfocean.drop (dfocean[dfocean.gn <= 2].index, inplace = True)

    dfocean['lol'] = dfocean.loc (axis = 1)[[x + '_l' for x in lxa]].values.tolist()

    dfocean = dfocean.sample (frac = 1).reset_index (drop = True)

    lol = [[y[max (ind - HISTORY_LENGTH, 0): ind] for y in x] for x, ind in zip (dfocean['lol'], dfocean['gn'])]

    trA = tf.ragged.constant (lol).to_tensor()
    trA = tf.transpose (trA, perm = [0, 2, 1])
    
    dfX = dfocean.loc(axis = 1)[lxd + ['gn']].copy (deep = True)
    dfY = dfocean.loc(axis = 1)[ ly].copy (deep = True)

    return trA, dfX, dfY

with open (fpLog, 'a') as flog:
    print ('fnPrepareData starts to prepare test data', file = flog)

trAtest,  dfXtest,  dfYtest  = fnPrepareData (dftest)

with open (fpLog, 'a') as flog:
    print ('Test data is ready', file = flog)
    print ('fnPrepareData starts to prepare train data', file = flog)

trAtrain, dfXtrain, dfYtrain = fnPrepareData (dftrain)

with open (fpLog, 'a') as flog:
    print ('Train data is ready', file = flog)
    print ('trAtrain: ', trAtrain.shape, file = flog)
    print ('dfXtrain: ', dfXtrain.shape, file = flog)
    print ('dfYtrain: ', dfYtrain.shape, file = flog)

ktrInputA = layers.Input (shape = (dfXtrain.shape[1], ), name = "INPUT_A")

ktrInputH = layers.Input (shape = (HISTORY_LENGTH, ROW_LENGTH), name = "INPUT_H") 

ktrMask = layers.Masking (mask_value = 0.)(ktrInputH)

ktrRNN = layers.LSTM (128)(ktrMask)
ktr = layers.Dense (units ='16', activation ='elu')(ktrRNN)
ktr = layers.BatchNormalization ()(ktr)

ktr = layers.concatenate ([ktrInputA, ktr], name = 'LAYER_CONCATENATE')
#ktr = layers.Dense (units ='256', activation ='elu')(ktr)
#ktr = layers.Dropout(0.1)(ktr)
ktr = layers.Dense(units ='256', kernel_regularizer = regularizers.l2(0.0001), activation = 'elu')(ktr)
ktr = layers.BatchNormalization ()(ktr)
#ktr = layers.Dropout(0.1)(ktr)
ktr = layers.Dense(units ='256', kernel_regularizer = regularizers.l2(0.0001), activation = 'elu')(ktr)
ktr = layers.BatchNormalization ()(ktr)
#ktr = layers.Dropout(0.05)(ktr)
ktr = layers.Dense(units ='256', kernel_regularizer = regularizers.l2(0.0001), activation = 'elu')(ktr)
ktr = layers.BatchNormalization ()(ktr)
#ktr = layers.Dropout(0.05)(ktr)
ktr = layers.Dense(units ='256', kernel_regularizer = regularizers.l2(0.0001), activation = 'elu')(ktr)
ktr = layers.BatchNormalization ()(ktr)
#ktr = layers.Dropout(0.05)(ktr)
ktrOut = layers.Dense(units ='20', kernel_regularizer = regularizers.l2(0.0001), activation = 'relu')(ktr)

mdSimple_h = keras.Model (inputs = [ktrInputA, ktrInputH], outputs = ktrOut)

mdSimple_h.summary()

iBatchSize = 64

#optimizer = tf.keras.optimizers.SGD (learning_rate = 0.001)#(lr_schedule)
optimizer = tf.keras.optimizers.Adam (learning_rate = 0.001)

mdSimple_h.compile (
    optimizer = optimizer, 
    loss = tf.keras.losses.MeanSquaredError(), 
#    loss = tf.keras.losses.MeanSquaredLogarithmicError(), 
    metrics = tf.keras.metrics.RootMeanSquaredError()
    )

history_h = mdSimple_h.fit (
    {"INPUT_A": dfXtrain.to_numpy(), "INPUT_H": trAtrain},
    dfYtrain.to_numpy(),
    epochs = 100,
    batch_size = iBatchSize, 
    validation_data = ({"INPUT_A": dfXtest.to_numpy(), "INPUT_H": trAtest}, dfYtest.to_numpy())
    )

fphist = Path ('history_p2.txt')
dfhist = pd.DataFrame (history_h.history)
dfhist.to_csv (fphist, index = False)


fnPrepareData starts to prepare train data
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 INPUT_H (InputLayer)           [(None, 32, 32)]     0           []                               
                                                                                                  
 masking (Masking)              (None, 32, 32)       0           ['INPUT_H[0][0]']                
                                                                                                  
 lstm (LSTM)                    (None, 128)          82432       ['masking[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 16)           2064        ['lstm[0][0]']                   
                                                   